Import data first: 

mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202201-divvy-tripdata.csv
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202202-divvy-tripdata.csv
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202203-divvy-tripdata.csv
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202204-divvy-tripdata.csv
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202205-divvy-tripdata.csv
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202206-divvy-tripdata.csv
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202207-divvy-tripdata.csv
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202208-divvy-tripdata.csv
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202209-divvy-tripdata.csv
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202210-divvy-tripdata.csv
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline 202211-divvy-tripdata.csv
mongoimport --type csv -d chicago_bikes -c divvy_ridedata --headerline  202212-divvy-tripdata.csv
mongoimport --type csv -d chicago_bikes -c weather_daily --headerline  weather_daily.csv

In [20]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import json
from pprint import pprint

In [21]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [22]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'chicago_bikes', 'classDB', 'config', 'epa', 'fruits_db', 'gardenDB', 'local', 'met', 'travel_db', 'uk_food']


In [23]:
# Assign the chicago_bikes database to a variable name
db = mongo['chicago_bikes']

In [24]:
# review the collections in our new database
print(db.list_collection_names())

['weather_daily', 'divvy_ridedata']


In [25]:
# review a document in the customer_list collection
print(db.divvy_ridedata.find_one())

{'_id': ObjectId('64e404c003505f880eefb45e'), 'ride_id': 'A6CF8980A652D272', 'rideable_type': 'electric_bike', 'started_at': '2022-01-10 08:41:56', 'ended_at': '2022-01-10 08:46:17', 'start_station_name': 'Glenwood Ave & Touhy Ave', 'start_station_id': 525, 'end_station_name': 'Clark St & Touhy Ave', 'end_station_id': 'RP-007', 'start_lat': 42.012763, 'start_lng': -87.6659675, 'end_lat': 42.01256011541, 'end_lng': -87.6743671152, 'member_casual': 'casual', 'started_at_date': '2022-01-10', 'started_at_time': '08:41:56'}


In [26]:
divvy_rides = db['divvy_ridedata']

In [27]:
divvy_rides.count_documents({})

5667717

In [19]:
documents = divvy_rides.find({})
bulk_updates = []

for document in documents:
    original_value = document["started_at"]
    parts = original_value.split(" ")
    date_part = parts[0]
    time_part = parts[1]
    bulk_updates.append(
        pymongo.UpdateOne(
            {"_id": document["_id"]},
            {
                "$set": {
                    "started_at_date": date_part,
                    "started_at_time": time_part
                }
            }
        )
    )

# Execute bulk write operations
divvy_rides.bulk_write(bulk_updates)


In [28]:
documents = divvy_rides.find({})
bulk_updates = []

for document in documents:
    original_value = document["ended_at"]
    parts = original_value.split(" ")
    date_part = parts[0]
    time_part = parts[1]
    bulk_updates.append(
        pymongo.UpdateOne(
            {"_id": document["_id"]},
            {
                "$set": {
                    "ended_at_date": date_part,
                    "ended_at_time": time_part
                }
            }
        )
    )

# Execute bulk write operations
divvy_rides.bulk_write(bulk_updates)

In [120]:
print(divvy_rides.find_one())
print(weather_daily.find_one())

{'_id': ObjectId('64e11f4d696f85250a48ff98'), 'ride_id': '81751A3186E59A6B', 'rideable_type': 'classic_bike', 'started_at': '2022-01-22 12:20:02', 'ended_at': '2022-01-22 12:32:06', 'start_station_name': 'Sheffield Ave & Fullerton Ave', 'start_station_id': 'TA1306000016', 'end_station_name': 'Damen Ave & Clybourn Ave', 'end_station_id': 13271, 'start_lat': 41.9256018819, 'start_lng': -87.6537080423, 'end_lat': 41.931931, 'end_lng': -87.677856, 'member_casual': 'member', 'date_column': '2022-01-22', 'time_column': '12:20:02', 'ended_at_date': '2022-01-22', 'ended_at_time': '12:32:06', 'afternoon_temp': 265.74, 'cloud_cover': 90.0, 'date': '2022-01-22', 'evening_temp': 266.09, 'max_temp': 269.31, 'max_windspeed': 3.58, 'min_temp': 265.58, 'morning_temp': 266.95, 'night_temp': 268.02, 'precipitation': 0.0}
{'_id': ObjectId('64e2bd76124daffb9a1d7f44'), 'date': '2022-01-01', 'cloud_cover': 90.0, 'precipitation': 0.18, 'min_temp': 273.83, 'max_temp': 278.76, 'morning_temp': 278.76, 'afternoo

In [ ]:
# Retrieve weather data
weather_data = weather_daily.find({})

# Prepare update operations
update_operations = []

for weather_document in weather_data:
    # Get the corresponding divvy_rides document based on the end_date
    started_at_date = weather_document['date']  # Adjust to the correct field name
    divvy_rides_document = divvy_rides.find_one({'started_at_date': started_at_date})

    # If a matching divvy_rides document is found, update it
    if divvy_rides_document:

        del weather_document['_id']

        update_operation = UpdateOne(
            {'_id': divvy_rides_document['_id']},
            {'$set': weather_document}
        )
        update_operations.append(update_operation)

# Execute bulk update operations
if update_operations:
    try:
        divvy_rides.bulk_write(update_operations)
        print("Bulk write completed successfully.")
    except Exception as e:
        print("Bulk write error:", e)